# <font color= "0C5FAD">Resultado final análisis archivo txt</font>

## Preanálisis

- Se debe añadir al archivo original raw un encabezado para que se tome como referencia del numero de columnas adecuadas en el momento de convertir los datos a un dataFrame de pandas. El formato es el siguiente ```DD/MM/AAAA, HH:MM a. m. - name surname: ‎archivo.jpg (archivo adjunto) info info info```
- Si el archivo proviene del index.js entonces el encabezado es como el siguiene ```time group name surname file info feature color branch country state temp```
- En el caso de que el archivo txt sea de whatsapp se debe **comentar** la siguiente línea ```df['time'] = df['time'] +" "+ df['group']```. En caso contrario, o sea que el archivo txt sea de node.js, se debe **descomentar**.
- Estos son ejemplos de formatos de escritura validos:
   120 # 39
 HOMBRE $90
 $390 # 36 #41 #40
 1850 tres pares total 15 pares
 $350 # 41 # 42 #43
 41 42 43

## Código para análisis

In [29]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, time, date
from weasyprint import HTML
# Código para crear un dataFrame pandas con el archivo txt raw
fecha = "2025-10-07"
df = pd.read_csv("2025-10-07.txt", sep=r'\s+' ,on_bad_lines='skip')
df['time'] = df['time'] +" "+ df['group'] #DESCOMENTAR esta línea si el archivo es de listening, comentar si es de whatsapp
#df = df.drop('group', axis=1)
##################################################################################################################
# Código para limpiar la información de los carácteres "," "$" "#" y espacios en blanco
# Después se identifica el tipo de dato y se hace la conversión dependiendo del tipo correcto int, date, time, str
def determinar_tipo_y_convertir(elemento):
    #Determina el tipo de dato y convierte el elemento al tipo correcto
    if pd.isna(elemento) or str(elemento).strip() == '':
     return "-"
    
    # LIMPIAR el elemento - quitar comas y espacios
    elemento_limpio = str(elemento).strip().rstrip(',').replace('$', '').replace('#', "").replace(';', "")
    
    # 1. Verificar si es FECHA (DD/MM/AAA,) - YA LIMPIO
    if re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', elemento_limpio):
        try:
            partes = elemento_limpio.split('/')
            dia, mes, año = int(partes[0]), int(partes[1]), int(partes[2])
            return datetime(año, mes, dia).date()
        except ValueError:
            pass
    # 1.1 Verificar hora HH:MMa. m.
    patron_12ham = re.match(r'^(\d{1,2}):(\d{2})\s*[ \s]*([ap])\.\s*[ \s]*m\.$', elemento_limpio, re.IGNORECASE)
    if patron_12ham:
        try:
            hora = int(patron_12ham.group(1))
            minuto = int(patron_12ham.group(2))
            periodo = patron_12ham.group(3).lower()  # 'a' o 'p'
            
            # Convertir de 12h a 24h
            if periodo == 'p' and hora != 12:
                hora += 12
            elif periodo == 'a' and hora == 12:
                hora = 0
            
            if 0 <= hora <= 23 and 0 <= minuto <= 59:
                return time(hora, minuto)
        except ValueError:
            pass
    # 2. Verificar si es HORA ([HH:MM:SS a. m./p. m.], etc.)
    patron_12h = re.match(r'^\[(\d{1,2}):(\d{2}):(\d{2})\s+([ap])\.\s*m\.\]$', elemento_limpio, re.IGNORECASE)
    if patron_12h:
        try:
            hora = int(patron_12h.group(1))
            minuto = int(patron_12h.group(2))
            segundo = int(patron_12h.group(3))
            periodo = patron_12h.group(4).lower()
            
            # Convertir de 12h a 24h
            if periodo == 'p' and hora != 12:
                hora += 12
            elif periodo == 'a' and hora == 12:
                hora = 0
            
            if 0 <= hora <= 23 and 0 <= minuto <= 59:
                return time(hora, minuto)
        except ValueError:
            pass
    # 3. Verificar si es NÚMERO ENTERO (usando elemento_limpio)
    if elemento_limpio.lstrip('-').isdigit() and elemento_limpio not in ['', '-']:
        numero = int(elemento_limpio)
        return numero
    
    # 4. Si no es ninguno de los anteriores, es TEXTO/NOMBRE (devolver el string original SIN COMA)
    return elemento_limpio

# CÓDIGO PRINCIPAL - UN SOLO BLOQUE
print("=== PROCESANDO DATAFRAME COMPLETO ===")
#######################################################################################################################################
# Crear nuevo DataFrame con los datos convertidos
df_limpio = df.map(determinar_tipo_y_convertir)

##Código para trabajar con el dataFrame limpio. Esto es identificar los datos que corresponden a precios
#y aquellos que corresponden a tallas para separarlos en columnas adicionales.

def extraer_y_separar_enteros(fila, indice_fila, umbral=50):
    enteros_mayores = []
    enteros_menores = []
    columnas_mayores = []
    columnas_menores = []
    
    for columna in df_limpio.columns:
        valor = fila[columna]
        if isinstance(valor, (int, np.integer)) and not isinstance(valor, bool):
            # Clasificar según el umbral
            if valor > umbral:
                enteros_mayores.append(valor)
                columnas_mayores.append(columna)
            else:
                enteros_menores.append(valor)
                columnas_menores.append(columna)
            
            # Reemplazar con cero en el DataFrame original
            df_limpio.at[indice_fila, columna] = 0
    
    return enteros_mayores, enteros_menores, columnas_mayores, columnas_menores

# Aplicar a todas las filas
resultados = []
for i in range(len(df_limpio)):
    fila = df_limpio.iloc[i]
    mayores, menores, cols_mayores, cols_menores = extraer_y_separar_enteros(fila, i)
    resultados.append((mayores, menores, cols_mayores, cols_menores))

# Agregar las nuevas columnas al DataFrame
df_limpio['precios'] = [resultado[0] for resultado in resultados]
df_limpio['tallas'] = [resultado[1] for resultado in resultados]
#df_limpio['col_precios'] = [resultado[2] for resultado in resultados]
#df_limpio['col_tallas'] = [resultado[3] for resultado in resultados]
#################################################################################################################################
# Línea para exportar el dataFrame a excel
#df_limpio.to_excel("revision-10-07.xlsx", index=False)
#Línea para mostrar el resultado final en consola

###############################################################################################################################
#Filtrar ventas del dia
def buscar_keyword(df, palabra, case_sensitive=False):
    mascara_total = pd.Series([False] * len(df))
    
    for columna in df.columns:
        try:
            mascara_columna = df_limpio[columna].astype(str).str.contains(
                palabra, case=case_sensitive, na=False, regex=False
            )
            mascara_total = mascara_total | mascara_columna
        except:
            continue
    
    return df_limpio[mascara_total]
resultado = buscar_keyword(df_limpio, 'Ventas_55')

#########################################################################################################
# Seleccionar solo las columnas que quieres
columnas_deseadas = ['time', 'precios', 'tallas']
df_seleccionado = resultado[columnas_deseadas]

# Desanidar y sumar las listas de precios
total_precios = sum(sum(lista) if isinstance(lista, list) else lista 
                   for lista in df_seleccionado['precios'])

# CONTAR los elementos en las listas de tallas
total_tallas = sum(len(lista) if isinstance(lista, list) else 1 
                  for lista in df_seleccionado['tallas'])

# Estilizar el DataFrame
styled_df = df_seleccionado.style.set_table_styles([
    {'selector': 'thead', 'props': [('background-color', '#4CAF50'), 
                                   ('color', 'white'),
                                   ('font-weight', 'bold')]},
    {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]},
    {'selector': 'tbody tr:hover', 'props': [('background-color', '#ddd')]},
    {'selector': 'table', 'props': [('border', '1px solid black'), 
                                   ('border-collapse', 'collapse')]},
    {'selector': 'th, td', 'props': [('border', '1px solid black'), 
                                    ('padding', '8px')]}
]).hide(axis='index')

# Crear HTML completo con ambos totales
html_content = f"""
<html>
<head>
<style>
    body {{ font-family: Arial, sans-serif; margin: 20px; }}
    .totales {{ 
        margin-top: 30px; 
        padding: 20px; 
        background-color: #f8f9fa; 
        border-radius: 8px;
        border-left: 5px solid #4CAF50;
    }}
</style>
</head>
<body>
    <div class='fecha'><strong>Fecha del Reporte:</strong>{fecha}</div>
    {styled_df.to_html()}
    <div class='totales'>
        <h3>Resumen Total:</h3>
        <p><strong>Total Precios:</strong> {total_precios}</p>
        <p><strong>Cantidad Total de Tallas:</strong> {total_tallas}</p>
    </div>
</body>
</html>
"""

# Exportar a PDF
HTML(string=html_content).write_pdf('informe_03.pdf')

=== PROCESANDO DATAFRAME COMPLETO ===


## Calcular la suma de todos los precios

In [13]:
# Sumar TODOS los enteros de TODAS las listas en la columna
suma_total_mayores = df_limpio['precios'].apply(
    lambda x: sum(x) if x else 0
).sum()

print(f"TOTAL: {suma_total_mayores}")

TOTAL: 4630


## Añadir filtro a dataframe con palabra específica

### Este código recorre las columnas del dataframe en busqueda de palabras clave requeridas

In [14]:
def buscar_keyword(df, palabra, case_sensitive=False):
    mascara_total = pd.Series([False] * len(df))
    
    for columna in df.columns:
        try:
            mascara_columna = df_limpio[columna].astype(str).str.contains(
                palabra, case=case_sensitive, na=False, regex=False
            )
            mascara_total = mascara_total | mascara_columna
        except:
            continue
    
    return df_limpio[mascara_total]

In [15]:
# Aplicar a tu DataFrame
resultado = buscar_keyword(df_limpio, 'Ventas_55')
resultado

,time,group,name,surname,file,info,feature,color,branch,country,state,temp,precios,tallas
0,10:11:00,a. m.],[Ventas_55],Jenny,Rodriguez:,-,-,-,-,-,-,-,[],[]
1,10:11:00,a. m.],[Ventas_55],Jenny,Rodriguez:,-,-,-,-,-,-,-,[],[]
2,10:11:00,a. m.],[Ventas_55],Jenny,Rodriguez:,HOMBRE,0,-,-,-,-,-,[90],[]
8,10:30:00,a. m.],[Ventas_55],Yolanda,Tenis:,0,tres,pares,total,0,pares,-,[1850],[15]
17,12:30:00,p. m.],[Ventas_55],Yolanda,Tenis:,0,,0,-,-,-,-,[120],[39]
31,12:55:00,p. m.],[Ventas_55],Yolanda,Tenis:,0,,0,-,-,-,-,[120],[42]
34,13:09:00,p. m.],[Ventas_55],Yolanda,Tenis:,0,,0,-,-,-,-,[125],[40]
63,14:44:00,p. m.],[Ventas_55],Yolanda,Tenis:,0,,0,0,0,-,-,[390],"[36, 41, 40]"
66,15:22:00,p. m.],[Ventas_55],Yolanda,Tenis:,0,paga,Diana,-,-,-,-,[120],[]
77,15:54:00,p. m.],[Ventas_55],Yolanda,Tenis:,0,,0,-,-,-,-,[230],[42]


### Código para contar elementos almacenados en columna tallas

In [21]:
def contador(df, nombre_columna):
    total_general = 0
    
    for lista in df[nombre_columna]:
        if isinstance(lista, list):
            for elemento in lista:
                if isinstance(elemento, int) and elemento < 50:
                    total_general += 1
    
    return total_general
    
total = contador(resultado, 'tallas')
print(total)

15


In [7]:
# Obtener el tipo de dato de una celda específica
fila = 0  # Índice de la fila
columna = 'group'  # Nombre de la columna

# Acceder al valor de la celda
valor_celda = df_limpio.at[fila, columna]

# Obtener el tipo de dato de la celda
tipo_dato_celda = type(valor_celda)

print(f"El valor de la celda es: {valor_celda}")
print(f"El tipo de dato de la celda es: {tipo_dato_celda}")


El valor de la celda es: 09:56:00
El tipo de dato de la celda es: <class 'datetime.time'>


### Generar archivo csv

In [16]:
resultado.to_excel("resultado.xlsx", index=False)

In [68]:
# Cuántos mensajes hay en cada hora del día
conteo_por_hora = df_limpio.groupby(df_limpio['hora'].apply(lambda x: x.hour)).size()
print(conteo_por_hora)

hora
11    12
12     7
13    10
14     7
15    13
17    14
18    17
dtype: int64


In [92]:
df_limpio.iloc[54,]

time                     17:44:00
group                      p. m.]
name       [Entra_sale_bodega_55]
surname                     Jenny
file                   Rodriguez:
info                            -
feature                         -
color                           -
branch                          -
country                         -
state                           -
temp                            -
precios                        []
tallas                         []
Name: 54, dtype: object

In [25]:
#df_limpio.at[52,"precios"]=[300,140]
print (df_limpio.at[52,"precios"])

[300, 140]


##  Para exportar un reporte en pdf

In [19]:
#import pandas as pd
from weasyprint import HTML
#import matplotlib.pyplot as plt

# Seleccionar solo las columnas que quieres
columnas_deseadas = ['time', 'precios', 'tallas']
df_seleccionado = resultado[columnas_deseadas]
# Estilizar el DataFrame
styled_df = df_seleccionado.style.set_table_styles([
    {'selector': 'thead', 'props': [('background-color', '#4CAF50'), 
                                   ('color', 'white'),
                                   ('font-weight', 'bold')]},
    {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]},
    {'selector': 'tbody tr:hover', 'props': [('background-color', '#ddd')]},
    {'selector': 'table', 'props': [('border', '1px solid black'), 
                                   ('border-collapse', 'collapse')]},
    {'selector': 'th, td', 'props': [('border', '1px solid black'), 
                                    ('padding', '8px')]}
]).hide(axis='index')

# Exportar a PDF
HTML(string=styled_df.to_html()).write_pdf('dataframe.pdf')